In [10]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

#scaler = MinMaxScaler()

In [11]:
file = 'train_preprocessed.csv'
df = pd.read_csv(f"../Data/{file}", low_memory=False)

y = df["Credit_Score"].to_numpy()

In [12]:
if file == 'train_preprocessed.csv':
    features_int = list(df.select_dtypes(include=["number"]).columns[-10:])
    features_cat = list(df.select_dtypes(include=["object"]).columns)
    
    features = features_int + features_cat
elif file == 'train_preprocessed_type_of_loan.csv':
    features = list(df.select_dtypes(include=["object"]).columns)

for feature in features:
    df[feature] = df[feature].astype('category')

df['Credit_Score'] = df['Credit_Score'].astype('category')

In [13]:
num = df.drop(columns='Credit_Score').select_dtypes(include=['number']).columns
cat = df.drop(columns='Credit_Score').select_dtypes(include=['category']).columns

In [14]:
df[cat] = SimpleImputer(strategy="most_frequent").fit_transform(df[cat])
df[num] = SimpleImputer(strategy="median").fit_transform(df[num])
for c in cat:
    df[c] = LabelEncoder().fit_transform(df[c])

In [15]:
from sklearn.feature_selection import mutual_info_regression, mutual_info_classif

MI_cat = mutual_info_classif(df[cat], df['Credit_Score'])

In [16]:
MI_cat_scores = pd.Series(dict(zip(cat, MI_cat))).sort_values(ascending=False)
features_cat = list(MI_cat_scores[MI_cat_scores > 0.10].index)
MI_cat_scores.to_dict()

{'Credit_Mix': 0.1813174166178555,
 'Payment_of_Min_Amount': 0.12459699090265586,
 'no Loan': 0.01711662039053108,
 'Auto Loan': 0.015921153139529354,
 'Mortgage Loan': 0.012895903870502545,
 'Not Specified': 0.01274919138639996,
 'Personal Loan': 0.0126485587297811,
 'Credit-Builder Loan': 0.0121998890590711,
 'Debt Consolidation Loan': 0.011624489294073115,
 'Student Loan': 0.011382843806934106,
 'Payday Loan': 0.010882788589779757,
 'Home Equity Loan': 0.009915964938570276,
 'Payment_Behaviour': 0.008539326467997155,
 'Month': 0.006193129053833513,
 'Occupation': 0.0003154078300844354}

In [24]:
list(MI_cat_scores.iloc[:-3].index)

Index(['Credit_Mix', 'Payment_of_Min_Amount', 'no Loan', 'Auto Loan',
       'Mortgage Loan', 'Not Specified', 'Personal Loan',
       'Credit-Builder Loan', 'Debt Consolidation Loan', 'Student Loan',
       'Payday Loan', 'Home Equity Loan'],
      dtype='object')

In [19]:
MI_cat

array([0.00619313, 0.00031541, 0.18131742, 0.12459699, 0.00853933,
       0.01711662, 0.01274919, 0.01219989, 0.01264856, 0.01162449,
       0.01138284, 0.01088279, 0.0128959 , 0.01592115, 0.00991596])

In [17]:
MI_num = mutual_info_regression(df[num], df['Credit_Score'])

MI_num_scores = pd.Series(dict(zip(num, MI_num))).sort_values(ascending=False)
features_num = list(MI_num_scores[MI_num_scores > 0.10].index)
print(MI_num_scores)

Monthly_Inhand_Salary       0.626965
Annual_Income               0.624436
Outstanding_Debt            0.611906
Total_EMI_per_month         0.568184
Interest_Rate               0.171900
Delay_from_due_date         0.123984
Num_Credit_Inquiries        0.111237
Num_Credit_Card             0.110593
Num_Bank_Accounts           0.104891
Credit_History_Age          0.101018
Changed_Credit_Limit        0.099511
Num_of_Loan                 0.085503
Num_of_Delayed_Payment      0.079281
Monthly_Balance             0.032744
Age                         0.028396
Amount_invested_monthly     0.009160
Credit_Utilization_Ratio    0.000000
dtype: float64


In [18]:
MI_num_scores.to_dict()

{'Monthly_Inhand_Salary': 0.6269652051783279,
 'Annual_Income': 0.6244361177387336,
 'Outstanding_Debt': 0.6119063802229476,
 'Total_EMI_per_month': 0.568184139196239,
 'Interest_Rate': 0.1718996955233738,
 'Delay_from_due_date': 0.12398352558209957,
 'Num_Credit_Inquiries': 0.11123686658605791,
 'Num_Credit_Card': 0.11059257866210892,
 'Num_Bank_Accounts': 0.10489057130167101,
 'Credit_History_Age': 0.10101796523620443,
 'Changed_Credit_Limit': 0.09951086245147245,
 'Num_of_Loan': 0.08550281134509952,
 'Num_of_Delayed_Payment': 0.07928052017931897,
 'Monthly_Balance': 0.03274398571529069,
 'Age': 0.028395734866581535,
 'Amount_invested_monthly': 0.00916024081308997,
 'Credit_Utilization_Ratio': 0.0}

In [56]:
from imblearn.pipeline import Pipeline 
from imblearn.over_sampling import RandomOverSampler

X_train, X_test, y_train, y_test = train_test_split(df[list(num) + list(cat)], y, test_size=0.2)

sampler = RandomOverSampler() 
#X_train, y_train = sampler.fit_resample(X_train, y_train)

'''
scaler = MaxAbsScaler()
X_train = scaler.fit_transform(X_train)#.toarray()
X_test = scaler.transform(X_test)#.toarray()'''